## Classification in PySpark
- This notebook covers developing a machine learning model in `PySpark` environment.
- Dataset used for analysis is [
SMS Spam Collection Data Set](https://archive.ics.uci.edu/ml/datasets/sms+spam+collection) taken from UCI Machine Learning Repository. A binary classification model is developed on this dataset to identify if the given text message a `Spam` message or a `Ham` message.
- Different data transformation and NLP techniques in PySpark environment are applied for Text cleaning and pre-processing.
- Classification model developement and tuning is performed under PySpark framework.

### 1. Installing Spark and JDK files to run a spark session on local computer

In [ ]:
# !ls

In [ ]:
# !apt-get update
# !apt-get install openjdk-8-jdk-headless -qq > /dev/null
# !wget -q http://archive.apache.org/dist/spark/spark-2.3.1/spark-2.3.1-bin-hadoop2.7.tgz
# !tar xf spark-2.3.1-bin-hadoop2.7.tgz
# !pip install -q findspark

In [ ]:
# !ls

In [ ]:
# importing necessary libraries to start a spark session

import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.3.1-bin-hadoop2.7"

import findspark
findspark.init()
from pyspark import SparkContext
sc = SparkContext.getOrCreate()

import pyspark
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate() 
spark

In [ ]:
# importing libraries necessary for data cleaning, pre-processin, ml model developement etc

import pandas as pd
import numpy as np
import nltk
import pyspark.sql.functions as f

from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from pyspark.sql.types import ArrayType, DataType, StringType
from pyspark.ml.feature import Imputer, StringIndexer, OneHotEncoder, VectorAssembler, StopWordsRemover, Tokenizer, HashingTF, IDF
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder
from pyspark.ml.evaluation import BinaryClassificationEvaluator,  MulticlassClassificationEvaluator

### 2. Data import and Target definition

In [ ]:
#importing model development data as a spark dataframe

df = spark.read.csv('/content/ClfData.csv', header=True, inferSchema=True)
print(df.show(5))

+----+--------------------+
|Flag|                SMS |
+----+--------------------+
| ham|Go until jurong p...|
| ham|Ok lar... Joking ...|
|spam|Free entry in 2 a...|
| ham|U dun say so earl...|
| ham|Nah I don't think...|
+----+--------------------+
only showing top 5 rows

None


In [ ]:
# adding an ID column to the dataset using pyspark functions

df = df.select('*').withColumn('ID', f.monotonically_increasing_id())
print('\nTop 5 rows')
print(df.show(5))
print('\nBottom 5 rows')
print(df.orderBy(f.desc('ID')).show(5))


Top 5 rows
+----+--------------------+---+
|Flag|                SMS | ID|
+----+--------------------+---+
| ham|Go until jurong p...|  0|
| ham|Ok lar... Joking ...|  1|
|spam|Free entry in 2 a...|  2|
| ham|U dun say so earl...|  3|
| ham|Nah I don't think...|  4|
+----+--------------------+---+
only showing top 5 rows

None

Bottom 5 rows
+----+--------------------+----+
|Flag|                SMS |  ID|
+----+--------------------+----+
| ham|Rofl. Its true to...|5573|
| ham|The guy did some ...|5572|
| ham|Pity, * was in mo...|5571|
| ham|Will Ã¼ b going t...|5570|
|spam|This is the 2nd t...|5569|
+----+--------------------+----+
only showing top 5 rows

None


In [ ]:
print('Data Shape : {} Rows - {} Columns'.format((df.count()), (len(df.columns))))
print('Class Distribution')
df.groupBy('Flag').count().orderBy('count').show()

Data Shape : 5574 Rows - 3 Columns
Class Distribution
+----+-----+
|Flag|count|
+----+-----+
|spam|  747|
| ham| 4827|
+----+-----+



In [ ]:
df.columns

['Flag', 'SMS ', 'ID']

In [ ]:
# renaming columns in a spark dataframe; the renaming commands are different from pandas commands

df = df.withColumnRenamed('SMS ', 'SMS')
df.columns

['Flag', 'SMS', 'ID']

In [ ]:
# Converting Target/Y column : Flag into a numeric Target/Y indicator using pyspark's StringIndexer
# 0 = Ham, 1 = Spam message

string_indexer = StringIndexer(inputCol='Flag', outputCol='Spam').fit(df)
df = string_indexer.transform(df)
df.show(5)

+----+--------------------+---+----+
|Flag|                 SMS| ID|Spam|
+----+--------------------+---+----+
| ham|Go until jurong p...|  0| 0.0|
| ham|Ok lar... Joking ...|  1| 0.0|
|spam|Free entry in 2 a...|  2| 1.0|
| ham|U dun say so earl...|  3| 0.0|
| ham|Nah I don't think...|  4| 0.0|
+----+--------------------+---+----+
only showing top 5 rows



### 3. Pre-processing using NLP techniques

In [ ]:
# creating a new dataframe to process and save text data

df_processed = df

In [ ]:
# converting text data into lower case and saving the results in a new column : 'SMS_Processed'

df_processed = df_processed.withColumn('SMS_Processed', f.lower(df_processed.SMS))
df_processed.take(5)

[Row(Flag='ham', SMS='Go until jurong point, crazy.. Available only in bugis n great world la e buffet... Cine there got amore wat...', ID=0, Spam=0.0, SMS_Processed='go until jurong point, crazy.. available only in bugis n great world la e buffet... cine there got amore wat...'),
 Row(Flag='ham', SMS='Ok lar... Joking wif u oni...', ID=1, Spam=0.0, SMS_Processed='ok lar... joking wif u oni...'),
 Row(Flag='spam', SMS="Free entry in 2 a wkly comp to win FA Cup final tkts 21st May 2005. Text FA to 87121 to receive entry question(std txt rate)T&C's apply 08452810075over18's", ID=2, Spam=1.0, SMS_Processed="free entry in 2 a wkly comp to win fa cup final tkts 21st may 2005. text fa to 87121 to receive entry question(std txt rate)t&c's apply 08452810075over18's"),
 Row(Flag='ham', SMS='U dun say so early hor... U c already then say...', ID=3, Spam=0.0, SMS_Processed='u dun say so early hor... u c already then say...'),
 Row(Flag='ham', SMS="Nah I don't think he goes to usf, he lives around

In [ ]:
# removing all HTML tags from text data and checking the results for first 5 rows

df_processed = df_processed.withColumn('SMS_Processed', f.regexp_replace(df_processed.SMS_Processed, r'<.*?>', ''))
df_processed.take(5)

[Row(Flag='ham', SMS='Go until jurong point, crazy.. Available only in bugis n great world la e buffet... Cine there got amore wat...', ID=0, Spam=0.0, SMS_Processed='go until jurong point, crazy.. available only in bugis n great world la e buffet... cine there got amore wat...'),
 Row(Flag='ham', SMS='Ok lar... Joking wif u oni...', ID=1, Spam=0.0, SMS_Processed='ok lar... joking wif u oni...'),
 Row(Flag='spam', SMS="Free entry in 2 a wkly comp to win FA Cup final tkts 21st May 2005. Text FA to 87121 to receive entry question(std txt rate)T&C's apply 08452810075over18's", ID=2, Spam=1.0, SMS_Processed="free entry in 2 a wkly comp to win fa cup final tkts 21st may 2005. text fa to 87121 to receive entry question(std txt rate)t&c's apply 08452810075over18's"),
 Row(Flag='ham', SMS='U dun say so early hor... U c already then say...', ID=3, Spam=0.0, SMS_Processed='u dun say so early hor... u c already then say...'),
 Row(Flag='ham', SMS="Nah I don't think he goes to usf, he lives around

In [ ]:
# removing all pucntuations from text data and checking the results for first 5 rows

df_processed = df_processed.withColumn('SMS_Processed', f.regexp_replace(df_processed.SMS_Processed, r'[^\w\s]', ''))
df_processed.take(5)

[Row(Flag='ham', SMS='Go until jurong point, crazy.. Available only in bugis n great world la e buffet... Cine there got amore wat...', ID=0, Spam=0.0, SMS_Processed='go until jurong point crazy available only in bugis n great world la e buffet cine there got amore wat'),
 Row(Flag='ham', SMS='Ok lar... Joking wif u oni...', ID=1, Spam=0.0, SMS_Processed='ok lar joking wif u oni'),
 Row(Flag='spam', SMS="Free entry in 2 a wkly comp to win FA Cup final tkts 21st May 2005. Text FA to 87121 to receive entry question(std txt rate)T&C's apply 08452810075over18's", ID=2, Spam=1.0, SMS_Processed='free entry in 2 a wkly comp to win fa cup final tkts 21st may 2005 text fa to 87121 to receive entry questionstd txt ratetcs apply 08452810075over18s'),
 Row(Flag='ham', SMS='U dun say so early hor... U c already then say...', ID=3, Spam=0.0, SMS_Processed='u dun say so early hor u c already then say'),
 Row(Flag='ham', SMS="Nah I don't think he goes to usf, he lives around here though", ID=4, Spam=0

In [ ]:
# removing unicodes from text data and checking the results for first 5 rows

df_processed = df_processed.withColumn('SMS_Processed', f.regexp_replace(df_processed.SMS_Processed, r'[^\x00-\x7F]+', ''))
df_processed.take(5)

[Row(Flag='ham', SMS='Go until jurong point, crazy.. Available only in bugis n great world la e buffet... Cine there got amore wat...', ID=0, Spam=0.0, SMS_Processed='go until jurong point crazy available only in bugis n great world la e buffet cine there got amore wat'),
 Row(Flag='ham', SMS='Ok lar... Joking wif u oni...', ID=1, Spam=0.0, SMS_Processed='ok lar joking wif u oni'),
 Row(Flag='spam', SMS="Free entry in 2 a wkly comp to win FA Cup final tkts 21st May 2005. Text FA to 87121 to receive entry question(std txt rate)T&C's apply 08452810075over18's", ID=2, Spam=1.0, SMS_Processed='free entry in 2 a wkly comp to win fa cup final tkts 21st may 2005 text fa to 87121 to receive entry questionstd txt ratetcs apply 08452810075over18s'),
 Row(Flag='ham', SMS='U dun say so early hor... U c already then say...', ID=3, Spam=0.0, SMS_Processed='u dun say so early hor u c already then say'),
 Row(Flag='ham', SMS="Nah I don't think he goes to usf, he lives around here though", ID=4, Spam=0

In [ ]:
# tokenizing the text data and saving the result as lists in a new column : SMS_Token

tokenizer = Tokenizer(inputCol='SMS_Processed', outputCol='SMS_Tokens')
df_processed = tokenizer.transform(df_processed)
df_processed.take(3)

[Row(Flag='ham', SMS='Go until jurong point, crazy.. Available only in bugis n great world la e buffet... Cine there got amore wat...', ID=0, Spam=0.0, SMS_Processed='go until jurong point crazy available only in bugis n great world la e buffet cine there got amore wat', SMS_Tokens=['go', 'until', 'jurong', 'point', 'crazy', 'available', 'only', 'in', 'bugis', 'n', 'great', 'world', 'la', 'e', 'buffet', 'cine', 'there', 'got', 'amore', 'wat']),
 Row(Flag='ham', SMS='Ok lar... Joking wif u oni...', ID=1, Spam=0.0, SMS_Processed='ok lar joking wif u oni', SMS_Tokens=['ok', 'lar', 'joking', 'wif', 'u', 'oni']),
 Row(Flag='spam', SMS="Free entry in 2 a wkly comp to win FA Cup final tkts 21st May 2005. Text FA to 87121 to receive entry question(std txt rate)T&C's apply 08452810075over18's", ID=2, Spam=1.0, SMS_Processed='free entry in 2 a wkly comp to win fa cup final tkts 21st may 2005 text fa to 87121 to receive entry questionstd txt ratetcs apply 08452810075over18s', SMS_Tokens=['free', 

In [ ]:
# removing stop words from SMS_Tokens and storing the result in a new column : SMS_Stop
# Updating SMS_Processed with stop words removed, and SMS_Tokens and SMS_Stop columns dropped

stop_words = stopwords.words('english')
remover = StopWordsRemover(stopWords=stop_words, inputCol='SMS_Tokens', outputCol='SMS_Stop')
df_processed = remover.transform(df_processed)
df_processed = df_processed.withColumn('SMS_Processed', f.concat_ws(' ', 'SMS_Stop'))
columns = ('SMS_Stop', 'SMS_Tokens')
df_processed = df_processed.drop(*columns)
df_processed.take(5)
#df1.join(df2, df1.ID == df2.ID).select(df1['*'], df2['col'])

[Row(Flag='ham', SMS='Go until jurong point, crazy.. Available only in bugis n great world la e buffet... Cine there got amore wat...', ID=0, Spam=0.0, SMS_Processed='go jurong point crazy available bugis n great world la e buffet cine got amore wat'),
 Row(Flag='ham', SMS='Ok lar... Joking wif u oni...', ID=1, Spam=0.0, SMS_Processed='ok lar joking wif u oni'),
 Row(Flag='spam', SMS="Free entry in 2 a wkly comp to win FA Cup final tkts 21st May 2005. Text FA to 87121 to receive entry question(std txt rate)T&C's apply 08452810075over18's", ID=2, Spam=1.0, SMS_Processed='free entry 2 wkly comp win fa cup final tkts 21st may 2005 text fa 87121 receive entry questionstd txt ratetcs apply 08452810075over18s'),
 Row(Flag='ham', SMS='U dun say so early hor... U c already then say...', ID=3, Spam=0.0, SMS_Processed='u dun say early hor u c already say'),
 Row(Flag='ham', SMS="Nah I don't think he goes to usf, he lives around here though", ID=4, Spam=0.0, SMS_Processed='nah dont think goes usf

In [ ]:
# Lemmetizing the text data to root word by combining NLTK and PySpark udf functinalities
# Updating the SMS_processed column with lemmatized words

lemmatizer = WordNetLemmatizer()
lemmatizer_udf = f.udf(lambda row: [lemmatizer.lemmatize(x) for x in row if x not in stop_words],
                       ArrayType(StringType()))
tokenizer = Tokenizer(inputCol='SMS_Processed', outputCol='SMS_Tokens')
df_processed = tokenizer.transform(df_processed)
df_processed = df_processed.withColumn('SMS_Lemmetize', lemmatizer_udf(f.col('SMS_Tokens')))
df_processed = df_processed.withColumn('SMS_Processed', f.concat_ws(' ', 'SMS_Lemmetize'))
columns = ('SMS_Lemmetize', 'SMS_Tokens')
df_processed = df_processed.drop(*columns)
df_processed.take(5)

[Row(Flag='ham', SMS='Go until jurong point, crazy.. Available only in bugis n great world la e buffet... Cine there got amore wat...', ID=0, Spam=0.0, SMS_Processed='go jurong point crazy available bugis n great world la e buffet cine got amore wat'),
 Row(Flag='ham', SMS='Ok lar... Joking wif u oni...', ID=1, Spam=0.0, SMS_Processed='ok lar joking wif u oni'),
 Row(Flag='spam', SMS="Free entry in 2 a wkly comp to win FA Cup final tkts 21st May 2005. Text FA to 87121 to receive entry question(std txt rate)T&C's apply 08452810075over18's", ID=2, Spam=1.0, SMS_Processed='free entry 2 wkly comp win fa cup final tkts 21st may 2005 text fa 87121 receive entry questionstd txt ratetcs apply 08452810075over18s'),
 Row(Flag='ham', SMS='U dun say so early hor... U c already then say...', ID=3, Spam=0.0, SMS_Processed='u dun say early hor u c already say'),
 Row(Flag='ham', SMS="Nah I don't think he goes to usf, he lives around here though", ID=4, Spam=0.0, SMS_Processed='nah dont think go usf l

In [ ]:
# Creating Term Frequency-Inverse Document Frequency embedding features on SMS_Processed data

# Tokenizing the lemmetized text data and saving results in new column : SMS_Tokens
tokenizer = Tokenizer(inputCol='SMS_Processed', outputCol='SMS_Tokens')
df_processed = tokenizer.transform(df_processed)

# Hashing the tokenized data and saving results in new column : TFFeatures
hashingTF = HashingTF(inputCol='SMS_Tokens', outputCol='TFFeatures', numFeatures=400)
df_processed = hashingTF.transform(df_processed)

# Creating TF-IDF features on hashed data and saving results in new column : IDFfeatures
idf = IDF(inputCol='TFFeatures', outputCol='IDFfeatures')
idfModel = idf.fit(df_processed)
df_processed = idfModel.transform(df_processed)

df_processed.take(3)

[Row(Flag='ham', SMS='Go until jurong point, crazy.. Available only in bugis n great world la e buffet... Cine there got amore wat...', ID=0, Spam=0.0, SMS_Processed='go jurong point crazy available bugis n great world la e buffet cine got amore wat', SMS_Tokens=['go', 'jurong', 'point', 'crazy', 'available', 'bugis', 'n', 'great', 'world', 'la', 'e', 'buffet', 'cine', 'got', 'amore', 'wat'], TFFeatures=SparseVector(400, {6: 1.0, 7: 1.0, 47: 1.0, 55: 1.0, 77: 1.0, 91: 1.0, 165: 1.0, 182: 1.0, 260: 1.0, 278: 1.0, 287: 1.0, 350: 1.0, 362: 1.0, 371: 1.0, 385: 1.0, 390: 1.0}), IDFfeatures=SparseVector(400, {6: 3.7509, 7: 3.0463, 47: 3.7819, 55: 2.883, 77: 2.7971, 91: 3.9532, 165: 3.3896, 182: 4.1262, 260: 3.1707, 278: 3.4499, 287: 4.583, 350: 3.7134, 362: 4.5152, 371: 4.583, 385: 4.8194, 390: 3.6988})),
 Row(Flag='ham', SMS='Ok lar... Joking wif u oni...', ID=1, Spam=0.0, SMS_Processed='ok lar joking wif u oni', SMS_Tokens=['ok', 'lar', 'joking', 'wif', 'u', 'oni'], TFFeatures=SparseVector

### 4. Model Development : Random Forest Classifier in PySpark

In [ ]:
# Selecting 3 columns for model development. TF-IDF features serve as X variables, and Spam as Y variable
df_model = df_processed.select(['ID', 'Spam', 'IDFfeatures'])
df_model.take(3)

[Row(ID=0, Spam=0.0, IDFfeatures=SparseVector(400, {6: 3.7509, 7: 3.0463, 47: 3.7819, 55: 2.883, 77: 2.7971, 91: 3.9532, 165: 3.3896, 182: 4.1262, 260: 3.1707, 278: 3.4499, 287: 4.583, 350: 3.7134, 362: 4.5152, 371: 4.583, 385: 4.8194, 390: 3.6988})),
 Row(ID=1, Spam=0.0, IDFfeatures=SparseVector(400, {20: 3.9439, 76: 3.4613, 84: 2.6991, 113: 4.2693, 278: 3.4499, 326: 1.7791})),
 Row(ID=2, Spam=1.0, IDFfeatures=SparseVector(400, {9: 3.8987, 30: 2.6346, 59: 4.9372, 70: 3.0615, 128: 3.7585, 140: 4.2822, 184: 4.3776, 233: 4.1152, 234: 4.0617, 235: 4.2316, 266: 3.0093, 273: 2.6676, 274: 4.6371, 279: 3.5956, 287: 9.166, 294: 4.0109, 308: 3.9721, 316: 3.6288, 346: 4.3323, 369: 3.2279, 389: 2.2459}))]

In [ ]:
# Splitting model development data into train and test data

df_train, df_test = df_model.randomSplit([0.85, 0.15], seed=21)
df_train.count(), df_test.count()

(4727, 847)

In [ ]:
# building base Random Forest model on Train data and getting model predicitons Test data
rf = RandomForestClassifier(labelCol='Spam', featuresCol='IDFfeatures', numTrees=200, maxDepth=5, seed=21)
rf_model = rf.fit(df_train)
rf_predictions = rf_model.transform(df_test)
rf_predictions.select('ID', 'Spam', 'prediction', 'probability').take(3)

[Row(ID=1, Spam=0.0, prediction=0.0, probability=DenseVector([0.9251, 0.0749])),
 Row(ID=7, Spam=0.0, prediction=0.0, probability=DenseVector([0.8256, 0.1744])),
 Row(ID=10, Spam=0.0, prediction=0.0, probability=DenseVector([0.9222, 0.0778]))]

In [ ]:
# evaluating base Random Forest model performance on Test data using AUROC and Accuracy metrics

rf_evaluator_roc = BinaryClassificationEvaluator(metricName='areaUnderROC', labelCol='Spam')
rf_evaluator_acc = MulticlassClassificationEvaluator(metricName='accuracy', labelCol='Spam')
auROC = round((rf_evaluator_roc.evaluate(rf_predictions)),2)
accuracy = round(((rf_evaluator_acc.evaluate(rf_predictions))*100),2)
print('Test auROC = {}'.format(auROC))
print('Test Accuracy = {}%'.format(accuracy))

Test auROC = 0.96
Test Accuracy = 88.19%


### 5. Hyperparameter Tuning in Pyspark

In [ ]:
# defining the hyparameter grid
parameter_grid = (ParamGridBuilder()
                  .addGrid(rf.maxDepth, [5,6,7])
                  .addGrid(rf.numTrees, [150, 200, 250])
                  .build())

# defining two model evaluation metrics, however the model will be tuned on AUROC values
rf_evaluator_roc = BinaryClassificationEvaluator(metricName='areaUnderROC', labelCol='Spam')
rf_evaluator_acc = MulticlassClassificationEvaluator(metricName='accuracy', labelCol='Spam')

# fitting model on train data and calculating predictions on test data
cv = CrossValidator(estimator=rf, evaluator=rf_evaluator_roc,
                    estimatorParamMaps=parameter_grid, numFolds=3, parallelism = 4)
rf_cv_model = cv.fit(df_train)
rf_cv_predictions = rf_cv_model.transform(df_test)
rf_predictions.select('ID', 'Spam', 'prediction', 'probability').take(3)

[Row(ID=1, Spam=0.0, prediction=0.0, probability=DenseVector([0.9251, 0.0749])),
 Row(ID=7, Spam=0.0, prediction=0.0, probability=DenseVector([0.8256, 0.1744])),
 Row(ID=10, Spam=0.0, prediction=0.0, probability=DenseVector([0.9222, 0.0778]))]

In [ ]:
# evaluating performance of tuned Random Forest model 
auROC = round((rf_evaluator_roc.evaluate(rf_cv_predictions)),2)
accuracy = round(((rf_evaluator_acc.evaluate(rf_cv_predictions))*100),2)
print('Test auROC = {}'.format(auROC))
print('Test Accuracy = {}%'.format(accuracy))

Test auROC = 0.97
Test Accuracy = 89.96%
